In [38]:
from math import log2
import torch
from torch import Tensor, sort


def num_swapped_pairs(ys_true: Tensor, ys_pred: Tensor) -> int:
    # допишите ваш код здесь
    pass

In [85]:
ys_true = torch.rand(4)
ys_pred = torch.rand(4)
ys_true, ys_pred

(tensor([0.6482, 0.8414, 0.2008, 0.2685]),
 tensor([0.6633, 0.2312, 0.6744, 0.8802]))

In [86]:
y_sort, indices = torch.sort(ys_true, descending=True)
y_sort, indices

(tensor([0.8414, 0.6482, 0.2685, 0.2008]), tensor([1, 0, 3, 2]))

In [87]:
ys_pred[indices]

tensor([0.2312, 0.6633, 0.8802, 0.6744])

In [88]:
num_correct_pairs = 0
ys_pred, indices = torch.sort(ys_pred, descending=True)
ys_true = ys_true[indices]
len_preds = len(ys_pred)
for i in range(len_preds):
    if i==len_preds-1:
        break
    for j in range(i+1, len_preds):
        if ys_true[i] > ys_true[j]:
            num_correct_pairs += 1
ys_true, ys_pred

(tensor([0.2685, 0.2008, 0.6482, 0.8414]),
 tensor([0.8802, 0.6744, 0.6633, 0.2312]))

In [89]:
num_correct_pairs

1

In [44]:
def compute_gain(y_value: float, gain_scheme: str) -> float: # зачем exp
    return y_value if gain_scheme=='const' else 2**y_value - 1

In [45]:
compute_gain(2, 'const')

2

In [46]:
compute_gain(2, 'exp')

3

In [103]:
def dcg(ys_true: Tensor, ys_pred: Tensor, gain_scheme: str) -> float:
    ys_pred, indices = torch.sort(ys_pred, descending=True)
    ys_true = ys_true[indices]

    sum_dcg = 0
    for i, y_true in enumerate(ys_true):
        gain = compute_gain(y_true, gain_scheme)
        sum_dcg += gain / log2(i + 2)
    return sum_dcg

In [111]:
ys_true = torch.FloatTensor([3.0, 1.0, 2.0, 1.0, 3.0])
ys_pred = torch.rand(5)
ys_true, ys_pred

(tensor([3., 1., 2., 1., 3.]),
 tensor([0.9683, 0.6117, 0.9045, 0.6148, 0.3598]))

In [112]:
preds_, indices = torch.sort(ys_pred, descending=True)
trues_ = ys_true[indices]
trues_, preds_

(tensor([3., 2., 1., 1., 3.]),
 tensor([0.9683, 0.9045, 0.6148, 0.6117, 0.3598]))

In [113]:
dcg(ys_true, ys_pred, 'const')

tensor(6.3531)

In [114]:
dcg(ys_true, ys_pred, 'exp')

tensor(12.5314)

In [115]:
def ndcg(ys_true: Tensor, ys_pred: Tensor, gain_scheme: str = 'const') -> float:
    case_dcg = dcg(ys_true, ys_pred, gain_scheme)
    ideal_dcg = dcg(ys_true, ys_true, gain_scheme)
    return case_dcg / ideal_dcg

In [116]:
ys_true, ys_pred

(tensor([3., 1., 2., 1., 3.]),
 tensor([0.9683, 0.6117, 0.9045, 0.6148, 0.3598]))

In [117]:
trues_, preds_

(tensor([3., 2., 1., 1., 3.]),
 tensor([0.9683, 0.9045, 0.6148, 0.6117, 0.3598]))

In [118]:
ndcg(ys_true, ys_pred, 'const')

tensor(0.9468)

In [119]:
ndcg(ys_true, ys_pred, 'exp')

tensor(0.9124)